### check Pandas Column contains a particular value

* 很多使用需要查看两个df直接的联系，例如某个列的值是否在另一个df中的某个列中有出现；
* 根据需求或者情形的不同，有多种不同的实现方式

In [1]:
import pandas as pd
import numpy as np

先看看最简单常用的关键字“in”

In [2]:
s = pd.Series(list('abc'))
s

0    a
1    b
2    c
dtype: object

最先想到使用的检查方法是：


In [3]:
'a' in s

False

结果却不是意料中的True，这是因为

in of a Series checks whether the value is in the **index**:

只检查是否在index中出现。

In [4]:
# 所以一下操作为True

1 in s

True

一直比较迂回点的办法是先转变为array再使用in来检查，例如借助unique()方法：

但是这种办法是比较蠢的，因为需要花费一定资源先计算unique()

In [5]:
s.unique()

array(['a', 'b', 'c'], dtype=object)

In [6]:
'a' in s.unique()

True

In [7]:
%timeit 'a' in s.unique()

The slowest run took 15.06 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 18.3 µs per loop


比这个好的办法当然是使用s.values:

In [8]:
%timeit 'a' in s.values

The slowest run took 12.81 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 4.01 µs per loop


以上的做法是判断单个值是否在column中出现，

很多时候在项目中是处理整个列，使用上述方法需要配合apply方法逐row判断

Series中有个很好用的method是 isin ,可以直接判断整个Series并返回相同shape的Series（bool dtype）

In [9]:
s1 = pd.Series(list('abcdads2er'))
s2 = pd.Series(list('dvadtqqtdd'))

In [10]:
s1.isin(s2)

0     True
1    False
2    False
3     True
4     True
5     True
6    False
7    False
8    False
9    False
dtype: bool

isin() 所带的参数不一定是Series，可以是所以list like的object 例如list set 等：

In [19]:
s1.isin(list('asdfasdf'))

0     True
1    False
2    False
3     True
4     True
5     True
6     True
7    False
8    False
9    False
dtype: bool

In [12]:
df = pd.DataFrame({'s1':s1, 's2': s2})

这个bool type的Series可以利用的地方还是很多的：
* 现在新的column
* 基于True Fasle取相关组合

In [13]:
df

,s1,s2
0,a,d
1,b,v
2,c,a
3,d,d
4,a,t
5,d,q
6,s,q
7,2,t
8,e,d
9,r,d


In [14]:
df['s1_in_s2'] = df.s1.isin(df.s2)

In [15]:
df

,s1,s2,s1_in_s2
0,a,d,True
1,b,v,False
2,c,a,False
3,d,d,True
4,a,t,True
5,d,q,True
6,s,q,False
7,2,t,False
8,e,d,False
9,r,d,False


In [23]:
df[df.s1.isin(list('test'))]

,s1,s2,s1_in_s2
6,s,q,False
8,e,d,False


In [24]:
df[~df.s1.isin(list('test'))]

,s1,s2,s1_in_s2
0,a,d,True
1,b,v,False
2,c,a,False
3,d,d,True
4,a,t,True
5,d,q,True
7,2,t,False
9,r,d,False
